# Setup

In [1]:
import sys
PYTHON_PATH = sys.executable

In [88]:
!$PYTHON_PATH -m pip install --upgrade bitsandbytes>=0.39.0 accelerate>=0.20.0
!$PYTHON_PATH -m pip install --upgrade transformers guidance 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached numpy-1.26.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 38.8 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0.dev0
    Uninstalling transform

In [2]:
!$PYTHON_PATH -m pip install --upgrade pandas==1.3.0 numpy

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
os.environ['OMP_NUM_THREADS'] = "32"

## Load model

In [3]:
from guidance import models, select, gen
model_name_or_path = "NousResearch/Meta-Llama-3-8B-Instruct"
# model_name_or_path = "NousResearch/Meta-Llama-3-70B-Instruct"
lm = models.Transformers(
    model_name_or_path, 
    device_map="auto", 
    load_in_8bit=True, 
    _attn_implementation='sdpa',
    echo=False
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import transformers
import torch 

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Get features from course

In [135]:
# import json
# with open('tmp/materials.json') as f:
#     materials = json.load(f)


In [137]:
import json
with open('tmp/edu_programs.json') as f:
    edu_programs = json.load(f)
program2url = {k: v['url'] for k,v in edu_programs.items()}
# program2url

### LLM tree

In [131]:
import json
with open('tmp/gb_program2url_texts.json', 'r') as f:
    program2url_text = json.load(f)

In [132]:
program2url_text

'Узнайте на бесплатном занятии\nсам готовить школьные презентации и рефераты\nбезопасно работать в интернете\nIT-направление для развития и полезного досуга\nСможет работать с презентациями, таблицами, графиками, текстом, изображениями. Узнает о цифровой гигиене в интернете, научится пользоваться электронной почтой и облачными хранилищами.\nПоможем выбрать дальнейший путь развития: ребёнок примерит роль гейм-дизайнера и иллюстратора-аниматора, сделает мини-игру и анимированное изображение.\nРебёнок научится находить полезные сайты, материалы, видеоролики и сохранять их в свою цифровую копилку. Ещё он узнает, как критически оценивать информацию и работать с ней.\nРебята успеют достичь результатов и собрать портфолио из проектов\nРебята разбирают новую тему и отрабатывают её на практике\nПроходят 1 или 2 раза в неделю в удобное для школьников время\nПреподаватель успевает уделить время каждому\n4 месяца — это оптимальная нагрузка для школьника. За это время он успеет освоить теорию и соб

In [149]:

def generate_prompt(program_id, course_title, program2url_text):
    program_desc = program2url_text[program_id]
    prompt = (
        f"Сгенерируй список из не более 10 навыков или технологий, "
        f"которые студент приобретет после успешного курса '{course_title}'. "
        "Каждый навык или технология должны быть сформулированы максимум в 5 словах. "
        f"Навыки должны быть релевантны содержанию курса. На сайте указана следующая информация по курсу: <text>{program_desc[:8192*2]}</text>.\n"
    )
    prompt += 'Приведи навыки и технологии нумерованным списком. Отвечай строго на русском языке.'
    return prompt, 8


program_id = '5455'
program = edu_programs[program_id]

course_title = program['name']

# Сгенерировать промпт на основе данных модуля
prompt, _= generate_prompt(program_id, course_title, program2url_text)
print(prompt)

Сгенерируй список из не более 10 навыков или технологий, которые студент приобретет после успешного курса 'Компьютерная грамотность'. Каждый навык или технология должны быть сформулированы максимум в 5 словах. Навыки должны быть релевантны содержанию курса. На сайте указана следующая информация по курсу: <text>Узнайте на бесплатном занятии
сам готовить школьные презентации и рефераты
безопасно работать в интернете
IT-направление для развития и полезного досуга
Сможет работать с презентациями, таблицами, графиками, текстом, изображениями. Узнает о цифровой гигиене в интернете, научится пользоваться электронной почтой и облачными хранилищами.
Поможем выбрать дальнейший путь развития: ребёнок примерит роль гейм-дизайнера и иллюстратора-аниматора, сделает мини-игру и анимированное изображение.
Ребёнок научится находить полезные сайты, материалы, видеоролики и сохранять их в свою цифровую копилку. Ещё он узнает, как критически оценивать информацию и работать с ней.
Ребята успеют достичь рез

In [150]:
from guidance import zero_or_more, regex
import guidance
import re
from tqdm import tqdm 

skills = []


# program = edu_programs['5455']

# course_title = edu_programs['5455']['package']
# # Структура модулей и курсов
# module_data = program['plan_edu']['modules'][0]
assistant_eos = '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
user_eos = '<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'

for program_id in tqdm(program2url_text, desc='programs'):
#     print(program_id)
    program = edu_programs[program_id]
    course_title = program['name']

    # Сгенерировать промпт на основе данных модуля
    prompt, max_programs = generate_prompt(program_id, course_title, program2url_text)
    messages = [
        {"role": "system", "content": "Ты очень полезный ассистен, который помогает извлекать полезные навыки из курсов"},
        {"role": "user", "content": prompt},
    ]


    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    prompt = prompt[:-len(assistant_eos)]
    prompt += "\n" + assistant_eos
    output = lm + prompt

    for i in range(1, max_programs+1):
        output += f"{i}. " + gen(stop='\n', max_tokens=50, name=f'skills', list_append=True) +'\n'
    skills += [[t.replace(assistant_eos.strip(), '') for t in output[f'skills']]]
#     break

output

In [151]:
skills

[['Создание презентаций',
  'Работа с таблицами и графиками',
  'Редактирование текста и изображений',
  'Безопасное использование интернета',
  'Управление электронной почтой',
  'Использование облачных хранилищ',
  'Создание анимированных изображений',
  'Разработка мини-игр'],
 ['Работа с Adobe Photoshop',
  'Создание иллюстраций',
  'Работа с цветом',
  'Адаптация иллюстраций для различных форматов',
  'Создание логотипов',
  'Работа с шрифтами',
  'Создание плакатов',
  'Создание дизайна коммуникаций'],
 ['Полигональное моделирование',
  'Скульптинг в Blender',
  'Рендеринг 3D-моделей',
  'Модификаторы в Blender',
  'Примитивы в 3D-моделировании',
  'Ноды в сценарии Blender',
  'Спецэффекты в 3D-анимации',
  'Геометрия в 3D-моделировании'],
 ['Создание ландшафта и постройки',
  'Программирование на языке Lua',
  'Создание спецэффектов и анимации',
  'Работа с переменными и типами данных',
  'Создание зданий и элементов интерьера',
  'Разработка игрового процесса и сценария',
  'Со

In [152]:
skills_cleaned = [[t.replace(assistant_eos.strip(), '') for t in l] for l in skills]

In [153]:
skills_cleaned

[['Создание презентаций',
  'Работа с таблицами и графиками',
  'Редактирование текста и изображений',
  'Безопасное использование интернета',
  'Управление электронной почтой',
  'Использование облачных хранилищ',
  'Создание анимированных изображений',
  'Разработка мини-игр'],
 ['Работа с Adobe Photoshop',
  'Создание иллюстраций',
  'Работа с цветом',
  'Адаптация иллюстраций для различных форматов',
  'Создание логотипов',
  'Работа с шрифтами',
  'Создание плакатов',
  'Создание дизайна коммуникаций'],
 ['Полигональное моделирование',
  'Скульптинг в Blender',
  'Рендеринг 3D-моделей',
  'Модификаторы в Blender',
  'Примитивы в 3D-моделировании',
  'Ноды в сценарии Blender',
  'Спецэффекты в 3D-анимации',
  'Геометрия в 3D-моделировании'],
 ['Создание ландшафта и постройки',
  'Программирование на языке Lua',
  'Создание спецэффектов и анимации',
  'Работа с переменными и типами данных',
  'Создание зданий и элементов интерьера',
  'Разработка игрового процесса и сценария',
  'Со

In [154]:
to_save = {
     program_id: [{'name': s} for s in skills_] 
     for (program_id, skills_) in zip(edu_programs, skills_cleaned)
}

In [155]:
import json
with open('tmp/skills_from_llm.json', 'w') as f:
    json.dump(to_save, f)

### debug

In [91]:
id_ = ["6156", "6144"]
if isinstance(id_, list):
    l = df[df.index.isin(id_)]
else:
    l = df[df.index == id_]
l

,name,url,Направление (тег),Специализация,Подвид,Тип устройства,"Технологии, инструменты",Уровень сложности,Класс,techs
6144,Разработчик,https://gb.ru/geek_university/developer,Информационные технологии,Разработчик — это максимально общее название с...,"Frontend,Backend,Fullstack,Ручное,Автоматизиро...","Десктоп,Веб,Mobile,Оборудование","Java,Python,JavaScript,C++,C,TCP/IP,SQL,NoSQL,...",Junior,Взрослый,"[Java, Python, JavaScript, C++, C, TCP/IP, SQL..."
6156,Инженер-аналитик,https://gb.ru/geek_university/developer/analyst,Информационные технологии,Инженер-аналитик — это специалист совмещающий ...,"Сбор и обработка данных,Визуализация данных,Ан...",,"SQL,NoSQL,Python,Pandas,Matplotlib,Seaborn,Pow...",Junior,Взрослый,"[SQL, NoSQL, Python, Pandas, Matplotlib, Seabo..."


In [86]:
df[df.index == id_]['url'][0]

'https://gb.ru/geek_university/developer'

In [280]:
df[df.index == id_]['Специализация'][0]

'Ландшафтный дизайн — это визуальный язык, позволяющий работать с природными средствами. Ландшафтный дизайнер сочетает три аспекта (архитектуру, ботанику и философию) в зависимости от того, как будет использоваться территория, которую он облагораживает., Дизайн среды'

In [213]:
df[df.index == '6239']['url'][0]

'https://gb.ru/geek_university/developer/programmer'

In [73]:
df[df.index == '6265']

,name,url,Направление (тег),Специализация,Подвид,Тип устройства,"Технологии, инструменты",Уровень сложности,Класс,techs
6265,Fullstack-разработчик,https://gb.ru/geek_university/developer/progra...,Информационные технологии,Fullstack-разработчики — универсальные програм...,Fullstack,,"JavaScript,HTML/CSS,PHP,Laravel,SQL,Git",Junior,Взрослый,"[JavaScript, HTML/CSS, PHP, Laravel, SQL, Git]"


In [185]:
df['Уровень сложности'].unique()

array(['Junior', 'Middle'], dtype=object)

In [186]:
df[df['Уровень сложности'] == 'Middle']

,name,Направление (тег),Специализация,Подвид,Тип устройства,"Технологии, инструменты",Уровень сложности,Класс
6304,Инженер по тестированию Мастер,Информационные технологии,QA-тестировщик или инженер по тестированию — э...,"Ручное,автоматизированное","Десктоп,Веб,Mobile,Оборудование,Умные устройства","Java,SQL,NoSQL,Allure,Spring,JUnit,Selenium,RE...",Middle,Взрослый
6261,Frontend-разработчик Мастер,Информационные технологии,Frontend-разработчик отвечает за внешний вид с...,Frontend,,"JavaScript,NodeJS,Vue.js,ReactJS,TypeScript,We...",Middle,Взрослый
6277,Программист Java Мастер,Информационные технологии,Java-разработчик занимается разработкой высоко...,Backend,,"Java,SQL,NoSQL,Spring,Ethernet,Linux,Git,LibGD...",Middle,Взрослый
6292,Инженер-программист Python Мастер,Информационные технологии,Python является самым популярным языком програ...,Backend,,"Python,SQL,NoSQL,Django,JavaScript,HTML/CSS,Git",Middle,Взрослый
6340,ИТ-инженер Data Science Мастер,Информационные технологии,Data Scientist анализирует большой объём данны...,,"Десктоп,Веб,Mobile,Оборудование","Python,SQL,Docker,NoSQL,ANNs,Keras,TensorFlow,...",Middle,Взрослый
6498,Графический дизайнер Мастер,Дизайн,"Графический дизайнер — это специалист, который...",,"Десктоп,Веб,Mobile,Оборудование,Умные устройства","Illustrator,Photoshop,InDesign",Middle,Взрослый
6506,Цифровой дизайнер Мастер,Дизайн,"Цифровой дизайнер, он же веб-дизайнер, придумы...",,,"Figma,Tilda,HTML,CSS,Adobe Photoshop",Middle,Взрослый
6644,Дата-сайентист в медицине Мастер,Информационные технологии,Data Scientist анализирует большой объём данны...,"Сбор и обработка данных,Анализ данных,Data Sci...","Оборудование,Умные устройства","Python,SQL,Docker,NoSQL,ANNs,Keras,TensorFlow,...",Middle,Взрослый
6652,DevOps-инженер Мастер,Информационные технологии,DevOps-инженеры контролируют все этапы создани...,Backend,Оборудование,"Ethernet,Unix,CI/CD,DNS,Docker,Kubernetes (K8s...",Middle,Взрослый
6540,UX/UI дизайнер Мастер,Дизайн,UX/UI-дизайнер проектирует взаимодействие поль...,,"Десктоп,Веб,Mobile,Оборудование,Умные устройства","UX/UI,Figma,Tilda,HTML,СSS,Adobe Photoshop",Middle,Взрослый
